# polytop
Python library for constructing polymer toplogies

### import the library and confirm version

In [4]:
from polytop import polytopVer
polytopVer()

0.9

In [ ]:
%%bash
jupyter nbconvert README.ipynb --to markdown